In [69]:


import os 
import pandas as pd
import numpy as np
import shutil
import random
import json

import sys
import numpy as np
import pandas as pd
from ctypes import CDLL
from  argparse import ArgumentParser
from timeit import default_timer as timer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import threadpoolctl
from threadpoolctl import threadpool_limits

class DummyEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, sparse):
        self.sparse = sparse

    def transform(self, X):
        ohe = OneHotEncoder(handle_unknown='ignore', sparse=self.sparse)
        return ohe.fit_transform(X)[:,1:]

    def fit(self, X, y=None, **fit_params):
        return self


class IdentityTransformer(BaseEstimator, TransformerMixin):
    def transform(self, input_array):
        return input_array

    def fit(self, X, y=None, **fit_params):
        return self



def dump_qr(dump_file_path, r):
    np.savetxt(dump_file_path, np.asarray(r), delimiter=',')


def transform_data(data, columns, cat_columns, sparse):
    transformer_a = []
    for column in columns:
        if column in cat_columns:
            transf_name = column + "_onehot"
            transformer_a.append((transf_name, DummyEncoder(sparse), [column]))
        else:
            transf_name = column + "_identity"
            transformer_a.append((transf_name, IdentityTransformer(), [column]))

    preprocessor = ColumnTransformer(transformers=transformer_a)
    one_hot_a = preprocessor.fit_transform(data)
    return one_hot_a


np.set_printoptions(threshold=sys.maxsize, precision=14)
pd.set_option('display.max_columns', 500)


In [70]:

db_names =["retailer_ohe", "favorita_ohe", "yelp_ohe"]
#db_names =["retailer", "favorita", "yelp"]
db_configs=  {
    "retailer": 
    {
        "db_config_path" : "/local/scratch/Figaro/figaro-code/system_tests/test_real_data/databases/usretailer/database_specs_usretailer_100.conf",
        "skip_attrs":  ["Population", "OccupiedHouseUnits", "SubCategory", "CategoryCluster"],
        "db_ohe_path": "/local/scratch/Figaro/data/retailer_ohe"
    },
    "favorita": 
    {
        "db_config_path" : "/local/scratch/Figaro/figaro-code/system_tests/test_real_data/databases/favorita/database_specs_favorita_100.conf",
        "skip_attrs": ["OnPromotion", "HolidayType", "Locale", "LocaleId", "State", "Cluster", "Family", "Perishable"],
        "db_ohe_path": "/local/scratch/Figaro/data/favorita_ohe"
    },
    "yelp": 
    {
        "db_config_path" : "/local/scratch/Figaro/figaro-code/system_tests/test_real_data/databases/yelp/database_specs_yelp_100.conf",
        "skip_attrs": ["ComplimentFunny"],
        "db_ohe_path": "/local/scratch/Figaro/data/yelp_ohe"
    },
    "retailer_ohe": 
    {
        "db_config_path" : "/local/scratch/Figaro/figaro-code/system_tests/test_real_data_pk/databases/usretailer/database_specs_usretailer_1_100.conf",
        "skip_attrs":  [ "Population", "White",  "Asian", "Pacific", "Black",  "MedianAge", "OccupiedHouseUnits", "HouseUnits", "Families", "Households", "HusbWife", "Males", "Females", "HouseholdChildren", "ZipC", "SubCategory", "Category", "CategoryCluster", "LocnC",  "DateIdC", "MaxTemp", "MinTemp", "Thunder"],
        "db_ohe_path": "/local/scratch/Figaro/data/retailer_ohe_ohe"
    },
    "favorita_ohe":
    {
        "db_config_path" : "/local/scratch/Figaro/figaro-code/system_tests/test_real_data_pk/databases/favorita/database_specs_favorita_1_100.conf",
        "skip_attrs":  ["OnPromotion", "HolidayType", "Locale", "LocaleId", "State", "Cluster", "Family", "Perishable"],
        "db_ohe_path": "/local/scratch/Figaro/data/favorita_ohe_ohe"
    },
    "yelp_ohe":
    {
        "db_config_path" : "/local/scratch/Figaro/figaro-code/system_tests/test_real_data_pk/databases/yelp/database_specs_yelp_1_100.conf",
        "skip_attrs":  [
                "CityId", "StateId", "Latitude", "Longitude", "StarsB", "ReviewCountB", "IsOpen",
                "StarsR", "Useful", "Funny", "Cool","UserIdC", "ReviewIdC","ComplimentFunny"],
        "db_ohe_path": "/local/scratch/Figaro/data/yelp_ohe_ohe"
    }

}
for db_name in db_names:
    db_config_path = db_configs[db_name]["db_config_path"]
    db_ohe_path = db_configs[db_name]["db_ohe_path"]
    skip_attrs = db_configs[db_name]["skip_attrs"]
    if os.path.exists(db_ohe_path):
         shutil.rmtree(db_ohe_path)
    os.makedirs(db_ohe_path)
    with open(db_config_path, 'r') as db_config_file:
        db_config_json = json.load(db_config_file)

    rels_json = db_config_json["database"]["relations"]

    skip_attrs = ["Population", "OccupiedHouseUnits", "SubCategory", "CategoryCluster"]
    os.rmdir(db_ohe_path)
    os.makedirs(db_ohe_path)

    for rel_json in rels_json:
        rel_name = rel_json["name"]
        attrs_json = rel_json["attributes"]
        data_path = rel_json["data_path"]
        print(rel_name)
        attrs = []
        attrs_without_dropped = []
        attrs_cat = []
        drop_attrs = []
        for attr_json in attrs_json:
            attr_name = attr_json["name"]
            attr_type = attr_json["type"]
            attrs.append(attr_name)
            if attr_name in skip_attrs:
                drop_attrs.append(attr_name)
            else:
                attrs_without_dropped.append(attr_name)
                if attr_type == "category":
                    attrs_cat.append(attr_name)
        
        print("attrs", attrs)
        print("cat_attrs", attrs_cat)
        print("drop_attrs",  drop_attrs)
        print("attrs_without_dropped",  attrs_without_dropped)

        table = pd.read_csv(data_path, names=attrs, 
            delimiter=",", header=None)
        print("dropping columns")
        table = table.drop(drop_attrs, axis="columns")
        print("ohe data")
        table_np = transform_data(table, attrs_without_dropped, attrs_cat, False)
        data_out_path = os.path.join(db_ohe_path, rel_name +".csv")
        print(data_out_path)
        np.savetxt(data_out_path, np.asarray(table_np), delimiter=',')
        


Sales
attrs ['Date', 'Store', 'Item', 'UnitSales', 'OnPromotion']
cat_attrs []
drop_attrs []
attrs_without_dropped ['Date', 'Store', 'Item', 'UnitSales', 'OnPromotion']
dropping columns
ohe data
/local/scratch/Figaro/data/favorita_ohe/Sales.csv
Oil
attrs ['Date', 'OilPrize']
cat_attrs []
drop_attrs []
attrs_without_dropped ['Date', 'OilPrize']
dropping columns
ohe data
/local/scratch/Figaro/data/favorita_ohe/Oil.csv
Holidays
attrs ['Date', 'HolidayType', 'Locale', 'LocaleId', 'Transferred']
cat_attrs ['Transferred']
drop_attrs []
attrs_without_dropped ['Date', 'HolidayType', 'Locale', 'LocaleId', 'Transferred']
dropping columns
ohe data
/local/scratch/Figaro/data/favorita_ohe/Holidays.csv
Transactions
attrs ['Date', 'Store', 'Transactions']
cat_attrs []
drop_attrs []
attrs_without_dropped ['Date', 'Store', 'Transactions']
dropping columns
ohe data
/local/scratch/Figaro/data/favorita_ohe/Transactions.csv
Stores
attrs ['Store', 'City', 'State', 'StoreType', 'Cluster']
cat_attrs ['City', 